<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#O-que-veremos?" data-toc-modified-id="O-que-veremos?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>O que veremos?</a></span></li><li><span><a href="#Requisitos-Básicos" data-toc-modified-id="Requisitos-Básicos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Requisitos Básicos</a></span><ul class="toc-item"><li><span><a href="#Teoria" data-toc-modified-id="Teoria-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Teoria</a></span></li><li><span><a href="#Técnico" data-toc-modified-id="Técnico-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Técnico</a></span></li></ul></li><li><span><a href="#Criando-Pipelines" data-toc-modified-id="Criando-Pipelines-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Criando Pipelines</a></span><ul class="toc-item"><li><span><a href="#Sem-Pipeline" data-toc-modified-id="Sem-Pipeline-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Sem Pipeline</a></span></li><li><span><a href="#Com-Pipeline" data-toc-modified-id="Com-Pipeline-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Com Pipeline</a></span></li></ul></li><li><span><a href="#Dissecando-com_pipeline.py" data-toc-modified-id="Dissecando-com_pipeline.py-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dissecando com_pipeline.py</a></span><ul class="toc-item"><li><span><a href="#Linhas-[10,-27]" data-toc-modified-id="Linhas-[10,-27]-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Linhas [10, 27]</a></span></li><li><span><a href="#Linhas-[29,-31]" data-toc-modified-id="Linhas-[29,-31]-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Linhas [29, 31]</a></span></li><li><span><a href="#Linhas-[34,-36]" data-toc-modified-id="Linhas-[34,-36]-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Linhas [34, 36]</a></span></li><li><span><a href="#Linhas-39,-41-e-46" data-toc-modified-id="Linhas-39,-41-e-46-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Linhas 39, 41 e 46</a></span></li></ul></li><li><span><a href="#Importante" data-toc-modified-id="Importante-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Importante</a></span></li><li><span><a href="#Esquema-Pipeline" data-toc-modified-id="Esquema-Pipeline-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Esquema Pipeline</a></span><ul class="toc-item"><li><span><a href="#Usando-o-.fit()" data-toc-modified-id="Usando-o-.fit()-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Usando o .fit()</a></span></li><li><span><a href="#Usando-.transform()" data-toc-modified-id="Usando-.transform()-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Usando .transform()</a></span></li><li><span><a href="#Usando-.fit_transform()" data-toc-modified-id="Usando-.fit_transform()-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Usando .fit_transform()</a></span></li></ul></li></ul></div>

# O que veremos? 

Em 99% dos modelos de Machine Learning criados(fonte eu mesmo), nós teremos que pré-processar e transformar os dados antes deles serem passados para um estimador do *scikit-learn*.<br>

O problema disso é que nós teremos muitas linhas de código com objetos que possuem os mesmos métodos .fit() e .transform() e um objeto final com .fit() e .predict(). Nós aprenderemos a colocar todos esses objetos dentro um único PIPELINE, assim usaremos apenas o PIPELINE.

# Requisitos Básicos 

## Teoria
Não há nenhuma teoria básica que seja condição necessária para que você consiga seguir com esse notebook.

## Técnico 
- Python | Requisitos em **02-Python-Packages-Libraries/0-Python**
    - Básico de Orientação a Objetos. Somente saber o que são objetos, classes, métodos, atributos etc. Nomenclatura e terminologia básicas
    - Imports de bibliotecas
- NumPy | Requisitos em **02-Python-Packages-Libraries/01-NumPy**
    - Arrays
    - Manipulação básica de arrays
- Pandas | Requisitos em **02-Python-Packages-Libraries/02-Pandas**
    - Leitura de dados
    - Dataframes e Series
    - Manipulação básica dos dois acima
- Scikit-Learn | Requisitos em **02-Python-Packages-Libraries/03-Scikit-Learn/00-Getting-Started**
    - Transformadores e pré-processadores
    - Fit e predict com estimadores

# Criando Pipelines 

## Sem Pipeline

Primeiro, vamos fazer um modelo sem usar o Pipeline e ver como o código ficaria.<br>

Vamos usar o dataset [Titanic do Kaggle](https://www.kaggle.com/c/titanic), entre no link para saber mais sobre a competição no próprio [Kaggle](https://www.kaggle.com/).<br>

Alguns pontos:
- A coluna PassengerId será usada como index
- Usaremos apenas as features Age e Fare, as duas serão nossa Design Matrix X
    - Age tem dados faltantes
- Survived é o nosso Target Values y
- Não iremos separar os dados em treino e teste. Nunca faça isso, aqui é somente um exemplo para mostrarmos o funcionamento geral do Pipeline, depois iremos mostrar com profundidade tudo o que temos.

In [1]:
"""
------sem_pipeline.py------

Script que mostra duas transformacoes:
    - Inserimos a media onde temos valores faltantes
    - Fazer o StandardScaler dos dados apos inserir os dados faltantes
Apos as duas transformacoes, nos usamos o estimador LogisticRegression
Repare na quantidade de passos que devemos fazer.
"""
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

PATH_FILE = 'data/titanic.csv'
USE_COLS = ['PassengerId', 'Survived', 'Age', 'Fare']

df_titanic = pd.read_csv(PATH_FILE, usecols=USE_COLS, index_col='PassengerId')

X = df_titanic.drop(columns='Survived')
y = df_titanic.loc[:, 'Survived']

print('Porcentagem de dados faltantes nas features')
print(f'{(X.isna().mean()*100).round(2)}')
print('Veja que a feature Age tem ~20% de dados faltantes')
print('--------------------------------------------------')

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()

# inserindo a media em dados faltantes
# vamos usar primeiro .fit() e depois .transform()
imputer.fit(X)
X_t_impute = imputer.transform(X)

# fazendo o scaling dos dados com StandardScaler
# apos inserirmos dados vazios
scaler.fit(X_t_impute)
X_t_scaler = scaler.transform(X_t_impute)

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_t_scaler, y)

# vamos ver a acuracia do modelo
# estamos fazendo isso nos mesmos 
# dados que treinamos, nunca faca isso
acc = log_reg.score(X_t_scaler, y)*100
print(f'\nAcc nos mesmos dados que treinamos: {acc:.2f}%')

Porcentagem de dados faltantes nas features
Age     19.87
Fare     0.00
dtype: float64
Veja que a feature Age tem ~20% de dados faltantes
--------------------------------------------------

Acc nos mesmos dados que treinamos: 65.43%


Veja a saída de cada transformação

In [2]:
# Design Matrix Original
X

,Age,Fare
PassengerId,,
1,22.0,7.2500
2,38.0,71.2833
3,26.0,7.9250
4,35.0,53.1000
5,35.0,8.0500
...,...,...
887,27.0,13.0000
888,19.0,30.0000
889,NaN,23.4500


In [3]:
# Design Matrix apos inserir a media
# onde tinhamos dados faltantes
X_t_impute

array([[22.        ,  7.25      ],
       [38.        , 71.2833    ],
       [26.        ,  7.925     ],
       ...,
       [29.69911765, 23.45      ],
       [26.        , 30.        ],
       [32.        ,  7.75      ]])

In [4]:
# Design Matrix apos inserir a media
# e fazer o standard scaler
X_t_scaler

array([[-0.5924806 , -0.50244517],
       [ 0.63878901,  0.78684529],
       [-0.2846632 , -0.48885426],
       ...,
       [ 0.        , -0.17626324],
       [-0.2846632 , -0.04438104],
       [ 0.17706291, -0.49237783]])

## Com Pipeline

Vamos reproduzir as mesmas transformações acima, mas dessa vez vamos usar o Pipeline.

In [5]:
"""
------com_pipeline.py------
Faremos exatamente as mesmas coisas que
fizemos em sem_pipeline.py, mas vamos fazer
uso do Pipeline.

Veja como reduzimos tudo a apenas uma linha, o Pipeline
ja treina e faz as transformacoes para a gente
"""
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

PATH_FILE = 'data/titanic.csv'
USE_COLS = ['PassengerId', 'Survived', 'Age', 'Fare']

df_titanic = pd.read_csv(PATH_FILE, usecols=USE_COLS, index_col='PassengerId')

X = df_titanic.drop(columns='Survived')
y = df_titanic.loc[:, 'Survived']

print('Porcentagem de dados faltantes nas features')
print(f'{(X.isna().mean()*100).round(2)}')
print('Veja que a feature Age tem ~20% de dados faltantes')
print('--------------------------------------------------')

imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
log_reg = LogisticRegression(random_state=42)

# instanciando o pipeline com os passos que queremos
pipeline = Pipeline([('ImputeMean', imputer), # primeiro ele insere a media
                     ('StandardScaler', scaler), # segundo ele faz o StandardScaler 
                     ('LogisticRegression', log_reg)]) # terceiro ele treina o estimador

# usamos o .fit() e ele ja faz todos os passos acima
pipeline.fit(X, y)

acc = pipeline.score(X, y)*100
print(f'\nAcc nos mesmos dados que treinamos: {acc:.2f}%')
print('--------------------------------------------------')

# podemos fazer previsoes apos dar .fit()
y_pred = pipeline.predict(X)
print('\n10 primeiras previsoes direto com o pipeline')
print(y_pred[:10])

Porcentagem de dados faltantes nas features
Age     19.87
Fare     0.00
dtype: float64
Veja que a feature Age tem ~20% de dados faltantes
--------------------------------------------------

Acc nos mesmos dados que treinamos: 65.43%
--------------------------------------------------

10 primeiras previsoes direto com o pipeline
[0 1 0 0 0 0 0 0 0 0]


Veja os passos(steps) do Pipeline que criamos.

In [6]:
print('Passos(steps) do Pipeline')
print(pipeline)

Passos(steps) do Pipeline
Pipeline(steps=[('ImputeMean', SimpleImputer()),
                ('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(random_state=42))])


Colocando a configurção display='diagram', conseguimos ver o Pipeline com uma representação em HTML quando usamos Jupyter-Notebook.

In [7]:
from sklearn import set_config
set_config(display='diagram')

pipeline

Pipeline(steps=[('ImputeMean', SimpleImputer()),
                ('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(random_state=42))])

# Dissecando com_pipeline.py

## Linhas [10, 27] 
Não vamos nos concentrar no intervalo de linhas [10, 27], já que elas são requisitos básicos para esse notebook. Use as referências indicadas no começo do notebook caso sinta alguma difilculdade em entender o que está acontecendo.

## Linhas [29, 31]

- 29
```python
imputer = SimpleImputer(strategy='mean')
```
    - instanciamos o SimpleImputer, que irá inserir a média onde temos dados faltantes
<br>
<br>
<br>
- 30
```python
scaler = StandardScaler()
```
    - instanciamos o StandardScaler, que irá aprender a média e o desvio padrão dos dados e depois fará a transformaçã $$z = \frac{(x - \bar{u})}{\bar{s}}$$
<br>
<br>
<br>    
- 31
```python
log_reg = LogisticRegression(random_state=42)
```
    - instanciamos o estimador LogisticRegression com o random_state em 42

## Linhas [34, 36]

```python
pipeline = Pipeline([('ImputeMean', imputer),
                     ('StandardScaler', scaler),
                     ('LogisticRegression', log_reg)])
```

Criamos o pipeline, veja que temos que passar uma lista de tuplas para dentro do **Pipeline()**, a tupla tem dois elementos
- Primeiro elemento: nome do passo, para que você possa acessar posteriormente
- Segundo elemento: o objeto que que fará a transformação ou estimador

Vamos adiantar aqui, mas caso queira acessar os passos, basta fazer igual o exemplo abaixo.

In [8]:
"""
Usando o atributo named_steps, nos
criamos um dicionario onde a chave eh o nome do passo
que voce deu e o valor eh o objeto em si.
"""
pipeline.named_steps

{'ImputeMean': SimpleImputer(),
 'StandardScaler': StandardScaler(),
 'LogisticRegression': LogisticRegression(random_state=42)}

In [9]:
"""
Para acessar, basta usar colchetes e passar
a chave que voce quiser
"""
pipeline.named_steps['StandardScaler']

StandardScaler()

In [10]:
"""
Nos conseguimos acessar os atributos dos
transformadores e ver o que ele aprendeu.
Veja este exemplo com o StandardScaler
"""
mean = pipeline.named_steps['StandardScaler'].mean_
std = pipeline.named_steps['StandardScaler'].scale_

print('Media e desvio padrao aprendidos pelo StandardScaler')
print(f'Media Age: {mean[0]:.2f} ||| Media Fare: {mean[1]:.2f}')
print(f'Desvio Padrao Age: {std[0]:.2f} ||| Desvio Padrao Fare: {std[1]:.2f}')

Media e desvio padrao aprendidos pelo StandardScaler
Media Age: 29.70 ||| Media Fare: 32.20
Desvio Padrao Age: 12.99 ||| Desvio Padrao Fare: 49.67


In [11]:
"""
Nos conseguimos acessar os atributos dos
transformadores e ver o que ele aprendeu.
Veja este exemplo com o SimpleImputer
"""
mean_age = pipeline.named_steps['ImputeMean'].statistics_[0]
mean_fare = pipeline.named_steps['ImputeMean'].statistics_[1]

print('Media aprendida pelo SimpleImputer')
print(f'Media Age: {mean_age:.2f}')
print(f'Media Fare: {mean_fare:.2f}')

Media aprendida pelo SimpleImputer
Media Age: 29.70
Media Fare: 32.20


## Linhas 39, 41 e 46 

- 39
```python
pipeline.fit(X, y)
```
    - Após criarmos o pipeline, damos um fit direto nos dados. Lembre-se, aqui estamos dando apenas um exemplo, esse .fit() deve ser nos dados de treino.
    - O pipeline já faz tudo de uma vez para você na sequência de passos que você colocou.
        - 1) ele insere a média calculada pelo SimpleImputer onde temos dados faltantes
        - 2) ele faz a transformação com o StandardScaler
        - 3) ele treina a LogisticRegression, esta é o nosso estimador para fazer previsões

- 41
```python
acc = pipeline.score(X, y)*100
```
    - Após darmos o .fit() com o pipeline, podemos usar o método score da própria LogisticRegression. Esse score é a acurácia dos dados que passarmos para ela, no nosso caso, X e y.

- 41
```python
y_pred = pipeline.predict(X)
```
    - Após dar o .fit(), podemos usar o pipeline para fazer previsões. Ele automaticamente dá um .transform() com o SimpleImputer e com o StandardScaler e por último usa o .predict() da LogisticRegression

# Importante

Nós não separamos em dados de treino e teste. Vamos bater sempre nessa tecla, o método .fit() sempre deve ser usado nos dados de treino e nunca nos de teste. Para os dados de teste, devemos usar apenas o método .transform(). Para não ficar decorando isso, basta usar o Pipeline e ele faz tudo para você.

#  Esquema Pipeline

Primeiro, nós instanciamos tudo que usaremos:
- Círculo: SimpleImputer(strategy='mean'), que calcula a média e insere onde temos dados faltantes
- Triângulo: StandardScaler(), que calcula a média $\bar{u}$ e o desvio padrão $\bar{s}$ dos dados e depois faz a transformação $$z = \frac{(x - \bar{u})}{\bar{s}}$$

![INSTANCE-OBJECTS](images/04_01_instance_objects.PNG)

Intanciamos o pipeline, passando para o Pipeline uma lista de tuplas, onde:
- Primeiro elemento da tupla: nome do passo, isso serve para acessarmos posteriormente
- Segundo elemento da tupla: objeto em si que fará a transformação ou .fit() e .predict()

![INSTANCE-PIPE](images/04_02_instance_pipe.PNG)

## Usando o .fit() 

Como sempre, o .fit() apenas aprende alguma coisa, no nosso caso:
- Passo **Impute**
    - aprende a média dos dados, 4.333
- Passo **Scaler**:
    - aprende a média dos dados, 4.333
    - apende o desvio padrão dos dados, 3.488

Conseguimos acessar os passos através do atributo .named_steps e verificar os parâmetros aprendidos de cada transformador.

![FIT-PIPE](images/04_03_fit_pipeline.PNG)

## Usando .transform()
Usando o .transform(), nós efetivamente fazemos as trasnformações com os parâmetros aprendidos pelo .fit()

- 1:
    - com a média 4.333 aprendida, o .transform() insere onde temos dados vazios(NaN)
- 2:
    - com a média $\bar{u}=4.333$ e o desvio padrão $\bar{s}=3.488$, fazemos a transformação $$z = \frac{(x - \bar{u})}{\bar{s}}$$
    em cada sample(linha) dos dados

![FIT-PIPE](images/04_04_transform_pipeline.PNG)

## Usando .fit_transform() 
Este método já faz o .fit(), ou seja, aprende os parâmetros com os dados e já aplica a transformação, tudo de uma vez. Geralmente é ele que usamos.